In [16]:
import warnings
warnings.filterwarnings('ignore') # :clown:

In [17]:
import pandas as pd
import numpy as np

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence

### Preparing The Data

In [20]:
from tensorflow import feature_column

In [21]:
SAMPLING_FREQUENCY = 1000
WINDOW_SIZE = SAMPLING_FREQUENCY # 1 second

In [22]:
# Custom function to map string labels to binary values
target_labels = ['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat']
def map(x, y):
    ecg_clean = tf.expand_dims(x['ECG_Clean'], axis=-1)  # Assuming ECG_Clean is a sequence of values

    # Convert string labels to binary values
    binary_label = tf.map_fn(lambda label: tf.cond(tf.reduce_any(tf.math.equal(label, target_labels)), lambda: 1, lambda: 0), y, dtype=tf.int32)

    return ecg_clean, binary_label

In [23]:
training_dataset=tf.data.experimental.make_csv_dataset(        
    './data/train.csv',
    batch_size=WINDOW_SIZE,
    shuffle=False,
    select_columns=['ECG_Clean', 'category'],
    label_name='category',
    num_epochs=1
)

val_dataset=tf.data.experimental.make_csv_dataset(        
    './data/val.csv',
    batch_size=WINDOW_SIZE,
    shuffle=False,
    select_columns=['ECG_Clean', 'category'],
    label_name='category',
    num_epochs=1
)

training_dataset = training_dataset.map(map)
val_dataset = val_dataset.map(map)
for x in training_dataset:
    print(x)
    break

(<tf.Tensor: shape=(1000, 1), dtype=float32, numpy=
array([[ 1.91395593e-05],
       [ 2.23040443e-05],
       [ 2.55021096e-05],
       [ 2.87627627e-05],
       [ 3.21119514e-05],
       [ 3.55695265e-05],
       [ 3.91477042e-05],
       [ 4.28556486e-05],
       [ 4.66994716e-05],
       [ 5.06791803e-05],
       [ 5.47978343e-05],
       [ 5.90676500e-05],
       [ 6.34855824e-05],
       [ 6.80623125e-05],
       [ 7.28131126e-05],
       [ 7.77410314e-05],
       [ 8.28460761e-05],
       [ 8.81221495e-05],
       [ 9.35509379e-05],
       [ 9.91126071e-05],
       [ 1.04778162e-04],
       [ 1.10517096e-04],
       [ 1.16301941e-04],
       [ 1.22099125e-04],
       [ 1.27878113e-04],
       [ 1.33616049e-04],
       [ 1.39286974e-04],
       [ 1.44872582e-04],
       [ 1.50357620e-04],
       [ 1.55732909e-04],
       [ 1.60992378e-04],
       [ 1.66125319e-04],
       [ 1.71134801e-04],
       [ 1.76013200e-04],
       [ 1.80754389e-04],
       [ 1.85353812e-04],
       [ 1.8

### Build the Model

In [29]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4634391/
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tfg.math.optimizer import levenberg_marquardt

model = Sequential()

# Add a hidden layer with 10 tan-sigmoidal neurons
model.add(Dense(10, input_dim=WINDOW_SIZE, activation='tanh'))

# Add the output layer with M neurons and softmax activation for classification
model.add(Dense(1, activation='softmax'))

# Compile the model using the Levenberg–Marquardt algorithm (you can also use other optimizers)
model.compile(optimizer=levenberg_marquardt(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                10010     
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
                                                                 
Total params: 10021 (39.14 KB)
Trainable params: 10021 (39.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# https://iopscience.iop.org/article/10.1088/1742-6596/1642/1/012021/pdf
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, Dense, GlobalAveragePooling1D, AveragePooling1D, Concatenate, MaxPooling1D

# def dense_block(x, growth_rate, num_layers):
#     for _ in range(num_layers):
#         # Batch Normalization
#         x = BatchNormalization()(x)
#         # ReLU activation
#         x = ReLU()(x)
#         # Convolution layer
#         x = Conv1D(growth_rate, kernel_size=3, padding='same', kernel_initializer='he_normal')(x)
#         # Concatenate with previous layers
#         x = Concatenate(axis=-1)([x, x])
#     return x

# def transition_layer(x, compression_factor):
#     # Batch Normalization
#     x = BatchNormalization()(x)
#     # Convolution layer
#     x = Conv1D(int(tf.keras.backend.int_shape(x)[-1] * compression_factor), kernel_size=1, padding='same', activation='relu', kernel_initializer='he_normal')(x)
#     # Average Pooling
#     x = AveragePooling1D(pool_size=2, strides=2, padding='same')(x)
#     return x

# def build_model(input_shape, growth_rate, dense_block_layers, compression_factor):
#     input_layer = Input(shape=input_shape)

#     # Initial convolution layer
#     x = BatchNormalization()(input_layer)
#     x = Conv1D(filters=2 * growth_rate, kernel_size=7, strides=2, padding='same', activation='relu', kernel_initializer='he_normal')(x)
#     x = MaxPooling1D(pool_size=3, strides=2, padding='same')(x)

#     # Layer 1
#     x = dense_block(x, growth_rate, dense_block_layers[0])
#     x = transition_layer(x, compression_factor)
                         
#     # Layer 2
#     x = dense_block(x, growth_rate, dense_block_layers[1])
#     x = transition_layer(x, compression_factor)
                         
#     # Layer 3
#     x = dense_block(x, growth_rate, dense_block_layers[2])
#     x = transition_layer(x, compression_factor)
                         
#     # Layer 4
#     x = dense_block(x, growth_rate, dense_block_layers[3])

#     # Global Average Pooling
#     x = GlobalAveragePooling1D()(x)

#     # Classification layer
#     x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = Dense(1, activation='sigmoid')(x)

#     model = tf.keras.Model(inputs=input_layer, outputs=x)

#     return model

# # Specify the input shape based on your data
# input_shape = (WINDOW_SIZE, 1)

# # Build and compile the model
# model = build_model(input_shape, growth_rate=12, dense_block_layers=[6, 8, 12, 6], compression_factor=0.5)
# model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# # Display the model summary
# model.summary()

In [26]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./checkpoints/cp-{epoch:04d}.ckpt',
                                                 save_weights_only=True,
                                                 verbose=1)

In [11]:
# Load the latest checkpoint if available
latest_checkpoint = tf.train.latest_checkpoint('./checkpoints/')
if latest_checkpoint is not None:
    print(f"Loading weights from {latest_checkpoint}")
    model.load_weights(latest_checkpoint)


Loading weights from ./checkpoints/cp-0001.ckpt



In [27]:
model.fit(
    training_dataset, 
    validation_data=val_dataset, 
    epochs=10, 
    callbacks=[
        cp_callback,
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/10


10000/10000 [==============================] - ETA: 0s - loss: 0.3541 - accuracy: 0.8594

KeyboardInterrupt: 

In [13]:
model.save('checkpoints/cp-model.h5')

In [14]:
test_dataset=tf.data.experimental.make_csv_dataset(        
    './data/test.csv',
    batch_size=WINDOW_SIZE,
    shuffle=False,
    select_columns=['ECG_Clean', 'category'],
    label_name='category',
    num_epochs=1
)

test_dataset = test_dataset.map(map)

In [15]:
results = model.evaluate(test_dataset)
results

194905/194905 [==============================] - 1771s 9ms/step - loss: 0.6173 - accuracy: 0.6971


[0.6172615885734558, 0.6970672607421875]